# Replicability

In [1]:
import pandas as pd
import numpy as np
from copairs.map import aggregate
from copairs.map import run_pipeline

In [2]:
data_level = "normalized_feature_select"

data_path = f"output/processed/{data_level}/combined.parquet"

df = pd.read_parquet(data_path)

In [3]:
df[[col for col in df.columns if col.startswith("Metadata_")]].sample(10)

,Metadata_Plate,Metadata_Well,Metadata_Plate_Map_Name,Metadata_cell_type,Metadata_line_ID,Metadata_line_condition,Metadata_line_source,Metadata_plating_density,Metadata_Object_Count,Metadata_Object_Count_inferred,Metadata_Site_Count
689,BR00132672,M18,NCP_2022_03_03_METADATA,neuron,44,deletion,human,3500,73.000,89.340000,3
151,BR00127194,G08,BR00127194,progen,19,control,human,10000,3818.000,3912.345731,9
663,BR00132672,L16,NCP_2022_03_03_METADATA,neuron,48,deletion,human,3500,95.000,104.592000,3
372,BR00127194,P13,BR00127194,progen,3,control,human,10000,3090.000,3193.194175,9
979,BR00132673,I20,NCP_2022_03_03_METADATA,neuron,27,deletion,human,3500,216.000,229.194000,3
1538,PE_PP_Plate2,A03,2022-08-07_batch_METADATA,astro,9,control,human,5000,182.000,250.544000,7
1195,BR_NCP_STEM_1,B20,BR_NCP_STEM_1,stem,16,control,human,10000,933.228,933.228000,9
1237,BR_NCP_STEM_1,D14,BR_NCP_STEM_1,stem,11,control,human,10000,624.456,624.456000,9
857,BR00132673,D18,NCP_2022_03_03_METADATA,neuron,43,deletion,human,3500,184.000,190.596000,3
804,BR00132673,B13,NCP_2022_03_03_METADATA,neuron,15,control,human,3500,431.000,478.218000,3


In [4]:
# extract meta and feat columns
meta_cols = [c for c in df.columns if c.startswith("Metadata")]
feat_cols = [c for c in df.columns if not c.startswith("Metadata")]
meta = df[meta_cols]
feats = df[feat_cols].values

# pos is what you are matching on
# sameby is what condition shall be the same (e.g., same perturbation, same target)
pos_sameby = [
    "Metadata_Plate",
    "Metadata_cell_type",
    "Metadata_line_ID",
    "Metadata_line_source",
]
# diffby is what condition shall be different (e.g., different plate)
pos_diffby = []

# neg is what you are matching against
# sameby is barely used unless you want to limit the sample, e.g., only against control in the same plate
neg_sameby = ["Metadata_Plate", "Metadata_cell_type"]
# diffby is what condition shall be different (e.g., control vs. non control)
neg_diffby = ["Metadata_line_ID"]
null_size = 1000
tech_result = run_pipeline(
    meta, feats, pos_sameby, pos_diffby, neg_sameby, neg_diffby, null_size
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

/Users/shsingh/mambaforge/envs/ncp/lib/python3.9/site-packages/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
# combine scores from samples with the same Metadata_Sample_Unique
tech_result_agg = aggregate(tech_result, pos_sameby, threshold=0.05)
tech_result_agg[["above_p_threshold", "above_q_threshold"]].value_counts()

above_p_threshold  above_q_threshold
False              False                287
Name: count, dtype: int64